### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [164]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [165]:
debug = False # if True, will print()

In [166]:

totalSchools = school_data_complete["school_name"].nunique()
totalStudents = school_data_complete["Student ID"].nunique()
totalBudget = school_data["budget"].sum()
avgMath = student_data["math_score"].mean()
avgRead = student_data["reading_score"].mean()

passingMath = len(school_data_complete[school_data_complete["math_score"] >= 70.0])
passingRead = len(school_data_complete[school_data_complete["reading_score"] >= 70.0])

hasMathScore = int(school_data_complete["math_score"].count())
hasReadScore = int(school_data_complete["reading_score"].count())

passingMathPercent = passingMath / hasMathScore
passingReadPercent = passingRead / hasReadScore
avgPassingPercent = (passingMathPercent + passingReadPercent)/2

if debug == True:
    print(f'Total schools: {totalSchools}')
    print(f'Total students: {totalStudents}')
    print(f'Total budget: {totalBudget}')

    print(f'Avg math score: {round(avgMath, 6)}')
    print(f'Avg reading score: {round(avgRead, 6)}')
    # (avg. math score + avg. reading score)/2
    print(f'Avg passing rate: {round((avgMath + avgRead)/2, 2)}')
    print(f'Passing math: {passingMath}')
    print(f'Has math score: {hasMathScore}')
    print(f'Has read score: {hasReadScore}')

    print(f'% passing math: {"%.6f%%" % (100 * passingMathPercent)}')
    print(f'% passing reading: {"%.6f%%" % (100 * passingReadPercent)}')
    print(f'% Overall Passing Rate: {"%.6f%%" % (100 * avgPassingPercent)}')

districtSum = pd.DataFrame(columns = [
    'Total Schools', 
    'Total Students', 
    'Total Budget', 
    'Average Math Score', 
    'Average Reading Score', 
    '% Passing Math', 
    '% Passing Reading', 
    '% Overall Passing Rate'
]) 
import locale
locale.setlocale( locale.LC_ALL, '' )

data = [{    
    'Total Schools':totalSchools, 
    'Total Students':"{:,.2f}".format(totalStudents), 
    'Total Budget':locale.currency(totalBudget, grouping=True), 
    'Average Math Score':round(avgMath, 6), 
    'Average Reading Score':round(avgRead, 6), 
    '% Passing Math': str('{:.6%}'.format(passingMathPercent)).replace('%', ''),
    '% Passing Reading': str("%.6f%%" % (100 * passingReadPercent)).replace('%', ''),
    '% Overall Passing Rate': str("%.6f%%" % (100 * avgPassingPercent)).replace('%', '')
}]

districtSum = districtSum.append(data, ignore_index = True)
from IPython.display import display, HTML
display(districtSum)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170.00","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [289]:
school_overall = school_data_complete.groupby(
    by=['school_name', 'type']
).agg({
    "Student ID": ['count'],
    "budget": ['min'],
    "math_score": ['mean'],
    "reading_score": ['mean']
})
school_overall.columns = ["students", "budget", "avg_math_score", "avg_reading_score"]
school_overall = school_overall.reset_index() # "harden" the column names

school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(
    by=['school_name', 'type']
).agg({
    "Student ID": ['count']
})
school_passing_math.columns = ["passing_math"]
school_passing_math = school_passing_math.reset_index() # "harden" the column names

school_passing_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(
    by=['school_name', 'type']
).agg({
    "Student ID": ['count']
})
school_passing_read.columns = ["passing_read"]
school_passing_read = school_passing_read.reset_index() # "harden" the column names

schoolOverall = pd.merge(school_overall, school_passing_math, on="school_name")
schoolOverall = pd.merge(schoolOverall, school_passing_read, on="school_name")
schoolOverall = schoolOverall.drop("type_x", 1)
schoolOverall.drop("type_y", axis=1, inplace=True)

schoolOverall["per_student_budget"] = (schoolOverall["budget"] / schoolOverall["students"]) 
schoolOverall["passed_math_perc"] = (schoolOverall["passing_math"] / schoolOverall["students"]) * 100
schoolOverall["passed_read_perc"] = (schoolOverall["passing_read"] / schoolOverall["students"]) * 100
schoolOverall["pass_rate"] = ((schoolOverall["passed_math_perc"] + schoolOverall["passed_read_perc"]) / 2)
schoolOverall.drop("passing_math", axis=1, inplace=True)
schoolOverall.drop("passing_read", axis=1, inplace=True)

schoolOverall.columns = [
    "School",
    "Total Students",
    "Total School Budget",
    "Average Math Score",
    "Average Reading Score",
    "School Type",
    "Per Student Budget",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]
# Reorder:
schoolOverall = schoolOverall[[
    "School",
    "School Type",
    "Total Students",
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]]

# Support for later:
schoolOverall["Total School Budget (raw)"] = schoolOverall["Total School Budget"]
schoolOverall["Per Student Budget (raw)"] = schoolOverall["Per Student Budget"]

schoolOverall["Total School Budget"] = ['${:,.2f}'.format(x) for x in schoolOverall["Total School Budget"]]
schoolOverall["Per Student Budget"] = ['${:,.2f}'.format(x) for x in schoolOverall["Per Student Budget"]]
schoolOverall

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Total School Budget (raw),Per Student Budget (raw)
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672,3124928,628.0
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652,1081356,582.0
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852,1884411,639.0
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308,1763916,644.0
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668,917500,625.0
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983,3022020,652.0
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391,248087,581.0
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171,1910635,655.0
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992,3094650,650.0
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270,585858,609.0


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [198]:
results = schoolOverall.nlargest(5, "% Overall Passing Rate")
results

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
12,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
13,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [199]:
results = schoolOverall.nsmallest(5, "% Overall Passing Rate")
results

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
10,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [254]:
ninth = school_data_complete[school_data_complete["grade"] == "9th"].groupby("school_name")["math_score"].mean()
tenth = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name")["math_score"].mean()
eleventh = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name")["math_score"].mean()
twelveth = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name")["math_score"].mean()

gby = school_data_complete.groupby(["school_name", "grade"])["math_score"].mean()
#pd.pivot_table(gby, values = 'avg_math_score', index=['school_name'], columns = 'grade').reset_index()
#pd.crosstab(gby["grade"])

grg = pd.merge(ninth, tenth, on="school_name", suffixes={"_09", "_10"})
grg = pd.merge(grg, eleventh, on="school_name", suffixes={"_prev11", "_11"})
grg = pd.merge(grg, twelveth, on="school_name", suffixes={"_11", "_12"})
grg.reset_index()
grg.columns = ["9th", "10th", "11th", "12th"]

# Optional:
grg["9th"] = ["{:,.2f}".format(x) for x in grg["9th"]]
grg["10th"] = ["{:,.2f}".format(x) for x in grg["10th"]]
grg["11th"] = ["{:,.2f}".format(x) for x in grg["11th"]]
grg["12th"] = ["{:,.2f}".format(x) for x in grg["12th"]]
grg

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [255]:
ninth = school_data_complete[school_data_complete["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
tenth = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
eleventh = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
twelveth = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

gby = school_data_complete.groupby(["school_name", "grade"])["reading_score"].mean()
#pd.pivot_table(gby, values = 'avg_reading_score', index=['school_name'], columns = 'grade').reset_index()
#pd.crosstab(gby["grade"])

grg = pd.merge(ninth, tenth, on="school_name", suffixes={"_09", "_10"})
grg = pd.merge(grg, eleventh, on="school_name", suffixes={"_prev11", "_11"})
grg = pd.merge(grg, twelveth, on="school_name", suffixes={"_11", "_12"})
grg.reset_index()
grg.columns = ["9th", "10th", "11th", "12th"]

# Optional:
grg["9th"] = ["{:,.2f}".format(x) for x in grg["9th"]]
grg["10th"] = ["{:,.2f}".format(x) for x in grg["10th"]]
grg["11th"] = ["{:,.2f}".format(x) for x in grg["11th"]]
grg["12th"] = ["{:,.2f}".format(x) for x in grg["12th"]]
grg

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [309]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#pd.cut(schoolOverall["Per Student Budget (raw)"], spending_bins, labels=group_names).head() 
schoolOverall2 = schoolOverall.copy()
schoolOverall2["Spending Ranges (per Student)"] = pd.cut(schoolOverall2["Per Student Budget (raw)"], spending_bins, labels=group_names)
schoolOverall2.reset_index()

sogby = schoolOverall2.groupby(
    by=["Spending Ranges (per Student)"]
).agg({
    "Average Math Score": ['mean'],
    "Average Reading Score": ['mean'],
    "% Passing Math": ['mean'],
    "% Passing Reading": ['mean'],
    "% Overall Passing Rate": ['mean']
})
sogby = sogby.reset_index() # "harden" the column names
sogby

,Spending Ranges (per Student),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,mean,mean,mean,mean,mean
0,<$585,83.455399,83.933814,93.460096,96.610877,95.035486
1,$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
2,$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
3,$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [285]:
#pd.cut(schoolOverall["Per Student Budget"], spending_bins, labels=group_names).head() 

## Scores by School Size

* Perform the same operations as above, based on school size.

In [314]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [316]:
schoolOverall3 = schoolOverall.copy()
schoolOverall3["School Size"] = pd.cut(schoolOverall3["Total Students"], size_bins, labels=group_names)
schoolOverall3.reset_index()

sogby2 = schoolOverall3.groupby(
    by=["School Size"]
).agg({
    "Average Math Score": ['mean'],
    "Average Reading Score": ['mean'],
    "% Passing Math": ['mean'],
    "% Passing Reading": ['mean'],
    "% Overall Passing Rate": ['mean']
})
sogby2 = sogby2.reset_index() # "harden" the column names
sogby2

,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,mean,mean,mean,mean,mean
0,Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
1,Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
2,Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [20]:
schoolOverall4 = schoolOverall.copy()
sogby3 = schoolOverall4.groupby(
    by=["School Size"]
).agg({
    "Average Math Score": ['mean'],
    "Average Reading Score": ['mean'],
    "% Passing Math": ['mean'],
    "% Passing Reading": ['mean'],
    "% Overall Passing Rate": ['mean']
})
sogby3 = sogby3.reset_index() # "harden" the column names
sogby3

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
